In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from pandas_datareader import DataReader
from pandas import Series, DataFrame

In [2]:
filedate = '20220407'

In [3]:
df_stock = pd.read_csv('data/stock_' + filedate + '.csv', index_col=0, header=[0,1], parse_dates=[0])

In [4]:
# 階層構造を逆にすることができます。今回は列方向で試します。
df_stock2 = df_stock.swaplevel('Attributes','Symbols',axis=1)

In [5]:
df_stock2.sort_values(by=['Symbols', 'Attributes'], axis=1, inplace=True)

In [6]:
# マルチインデックスのカラム名を取得する
list = df_stock2.columns.get_level_values('Symbols')

In [7]:
# RSIの計算
def rsi(df):
    # 前日との差分を計算
    df_diff = df[i,"Close"].diff(1)

    # 計算用のDataFrameを定義
    df_up, df_down = df_diff.copy(), df_diff.copy()
    
    # df_upはマイナス値を0に変換
    # df_downはプラス値を0に変換して正負反転
    df_up[df_up < 0] = 0
    df_down[df_down > 0] = 0
    df_down = df_down * -1
    
    # 期間03でそれぞれの平均を算出
    df_up_sma03 = df_up.rolling(window=3, center=False).mean()
    df_down_sma03 = df_down.rolling(window=3, center=False).mean()

    # RSIを算出
    df[i, "RSI"] = 100.0 * (df_up_sma03 / (df_up_sma03 + df_down_sma03))

    return df

for i in list: 
    df_stock2 = rsi(df_stock2)

C:\Users\ricew\AppData\Local\Temp/ipykernel_256080/2502226233.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i, "RSI"] = 100.0 * (df_up_sma03 / (df_up_sma03 + df_down_sma03))


In [8]:
# High-Lowの計算結果を新たな列として追加
for i in list: 
    df_stock2[i, 'diff'] = df_stock2[i, 'High'] - df_stock2[i, 'Low']
    df_stock2[i, 'eighty'] = df_stock2[i, 'High'] - df_stock2[i, 'diff'] * 0.2
    df_stock2[i, 'twenty'] = df_stock2[i, 'Low'] + df_stock2[i, 'diff'] * 0.2
    df_stock2[i, 'RSI_diff'] = df_stock2[i,"RSI"].diff(1)

C:\Users\ricew\AppData\Local\Temp/ipykernel_256080/2650847954.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_stock2[i, 'diff'] = df_stock2[i, 'High'] - df_stock2[i, 'Low']
C:\Users\ricew\AppData\Local\Temp/ipykernel_256080/2650847954.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_stock2[i, 'eighty'] = df_stock2[i, 'High'] - df_stock2[i, 'diff'] * 0.2
C:\Users\ricew\AppData\Local\Temp/ipykernel_256080/2650847954.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fra

In [9]:
df_stock2.sort_values(by=['Symbols', 'Attributes'], axis=1, inplace=True)

In [10]:
# 日付の比較Datetimeから最後の日のみを取得
df_stock3 = df_stock2.tail(1)

In [11]:
df_stock3 = df_stock3.T

In [12]:
df_stock3 = df_stock3.pivot_table(index=['Symbols'], columns=['Attributes'])

In [13]:
df_stock3 = df_stock3.droplevel(0, axis=1)

In [14]:
# 日付の比較Datetimeから日付のみを取得して比較
df_stock4 = df_stock3[(df_stock3['RSI'] < 30) &
                 (df_stock3['Open'] >= df_stock3['eighty']) &
                 (df_stock3['Close'] <= df_stock3['twenty'])
                 ]

In [15]:
# 日付の比較Datetimeから日付のみを取得して比較
df_stock5 = df_stock3[(df_stock3['RSI'] > 70) &
                 (df_stock3['Close'] >= df_stock3['eighty']) &
                 (df_stock3['Open'] <= df_stock3['twenty'])
                 ]

In [16]:
df_stock4.to_csv('data/pin_80_20_買い_' + filedate + '.csv')

In [17]:
df_stock5.to_csv('data/pin_80_20_売り_' + filedate + '.csv')